# Import modules

In [220]:
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from dateutil import parser

# Set display options

In [221]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
desired_width = 320
pd.set_option('display.width', desired_width)

# Import Data

In [222]:
currPath = '/home/vmuser/Documents/analysis'

dataManager = 'ManagerStats.csv'
dataScores = 'MLBScores.csv'
dataSalaries = 'Salaries.csv'
dataTeamStats = 'TeamStats.csv'
dataDates = 'DatesbyTeam.csv'

# Descriptive Statistics of Game Outcomes

In [223]:
dc = pd.read_csv(currPath + '/' + dataScores, low_memory=False)

dc.describe().round(2)

,Season,Rk,Gm,Date,Unnamed: 4,...,D/N,Attendance,Streak,Streak1,PriorStreak
count,54803,54803,54803,54803,53454,...,54803,54803,54803,54803,54016
unique,13,164,164,1278,1,...,3,19044,34,34,35
top,2013,Rk,Gm#,Date,boxscore,...,N,Attendance,-,-1,1
freq,4982,1349,1349,1349,53454,...,36064,1349,13170,13170,13304


# Descriptive Statistics of Managerial Statistics

In [224]:
dmgr = pd.read_csv(currPath + '/' + dataManager, low_memory=False)

dmgr.describe().round(2)

,Season,Yrs,Prior Seasons Managed,From,To,...,WSwon,PennWon,ASG,playoff per season,career playoffs per season
count,330.00,330.00,330.00,330.00,330.00,...,330.00,330.00,330.00,330.00,330.00
mean,2009.00,11.46,7.52,1999.51,2012.27,...,0.54,1.03,0.98,0.21,0.25
std,3.17,7.48,7.23,9.67,2.41,...,0.96,1.58,1.53,0.24,0.20
min,2004.00,1.00,0.00,1973.00,2004.00,...,0.00,0.00,0.00,0.00,0.00
25%,2006.00,6.00,2.00,1995.00,2011.00,...,0.00,0.00,0.00,0.00,0.02
50%,2009.00,11.00,6.00,2002.00,2014.00,...,0.00,0.00,0.00,0.17,0.25
75%,2012.00,15.00,11.00,2007.00,2014.00,...,1.00,1.00,1.00,0.36,0.38
max,2014.00,33.00,32.00,2014.00,2014.00,...,4.00,6.00,6.00,1.00,1.00


# Descriptive Statistics of Team Payroll

In [225]:
dpay = pd.read_csv(currPath + '/' + dataSalaries, low_memory=False)

dpay.describe().round(2)

,Year,Start Pay,End Pay,End Rank,OWAR,DWAR,PWAR,WAR
count,330.00,3.300000e+02,3.300000e+02,330.00,330.00,330.00,330.00,330.00
mean,2009.00,9.009438e+07,9.511560e+07,15.48,19.68,-0.00,13.67,33.34
std,3.17,3.923318e+07,4.138032e+07,8.67,6.65,3.81,7.03,10.64
min,2004.00,1.499850e+07,2.112433e+07,1.00,0.50,-12.40,-5.80,3.80
25%,2006.00,6.329638e+07,6.920405e+07,8.00,14.82,-2.40,8.83,25.52
50%,2009.00,8.518380e+07,8.981920e+07,15.00,19.65,0.25,14.00,33.55
75%,2012.00,1.069286e+08,1.135385e+08,23.00,23.80,2.70,18.17,41.00
max,2014.00,2.293359e+08,2.572834e+08,30.00,40.00,11.10,37.20,59.90


# Descriptive Statistics of Average Age

In [226]:
dage =  pd.read_csv(currPath + '/' + dataTeamStats, low_memory=False)

dage.describe().round(2)

,Season,NumBat,BatAge,PerBat,NumP,PAge,PerP,AvgAge
count,330.00,330.00,330.00,330.00,330.00,330.00,330.00,330.00
mean,2009.00,46.07,28.87,0.66,23.52,28.61,0.34,28.79
std,3.17,4.65,1.33,0.02,3.49,1.38,0.02,1.20
min,2004.00,36.00,25.60,0.61,15.00,25.70,0.28,25.70
25%,2006.00,43.00,27.90,0.65,21.00,27.70,0.32,28.03
50%,2009.00,46.00,28.80,0.66,23.00,28.40,0.34,28.58
75%,2012.00,49.00,29.70,0.68,25.75,29.37,0.35,29.43
max,2014.00,64.00,33.50,0.72,40.00,34.20,0.39,33.04


# Team-game dates 2004-2014

In [227]:
dd = pd.read_csv(currPath + '/' + dataDates, low_memory=False)
dd['fullDate'] = dd.apply(lambda x: parser.parse(x['Date']),axis=1)
dd = dd.sort_values(by=['Season'], ascending=[1])

print("Descriptive statistics")
dd.describe().round(2)

Descriptive statistics


,Season
count,67650.00
mean,2009.00
std,3.16
min,2004.00
25%,2006.00
50%,2009.00
75%,2012.00
max,2014.00


# Create scores data set and import dates

In [228]:
dc = pd.read_csv(currPath + '/' + dataScores, low_memory=False)
dc = dc[dc['Date'] != 'Date'] #Remove extra heading rows
dc['Date'] = dc['Date'] + ', ' + dc['Season']
dc['Date'] = dc.apply(lambda x: parser.parse(x['Date']), axis=1)
dc['isthome'] = dc.apply(lambda x: 1 if x['Symbol']!="@" else 0, axis=1)

# Rename headers and convert to values to floats

In [229]:
dc = dc.rename(columns={'Visitor': 'team', 'Home': 'opp', 'Visitor W/L': 'gameWL', 'Runs Visitor': 'teamRuns', 'Runs Home': 'oppRuns', 'D/N': 'DN', 'Gm': 'gn'})
dc = dc.drop(['Unnamed: 4', 'W-L', 'Win', 'Loss', 'Save', 'Rk', 'gameWL', 'Inn', 'GB', 'Rank', 'Streak', 'Time', 'Symbol'], axis=1)
dc[['Season', 'teamRuns', 'oppRuns', 'gn']] = dc[['Season', 'teamRuns', 'oppRuns', 'gn']].astype(float) 

# Determine winners and losers by run differential

In [264]:
dc['druns'] = dc['teamRuns'] - dc['oppRuns'] #difference in runs
dc['twin'] = dc.apply(lambda x: 1 if x['druns'] > 0 else 0, axis=1) #winner indicator variable
dc['tlos'] = dc.apply(lambda x: 1 if x['druns'] < 0 else 0, axis=1) #loser indicator variable

dc = dc.sort_values(by=['Season', 'team', 'gn'], ascending=[1, 1, 1])
dc = dc.set_index(['Season', 'team'])

# Attain pre-game records

In [231]:
dc['tws'] = dc.groupby(level=['Season', 'team'])['twin'].cumsum() - dc['twin']
dc['tls'] = dc.groupby(level=['Season', 'team'])['tlos'].cumsum() - dc['tlos']
dc['twper'] = dc['tws'] / (dc['tws'] + dc['tls'])
dc['tgameday'] = 1
dc = dc.reset_index()

# Loop to create individual data by season/team

In [232]:
season_list = dc['Season'].unique()  ###Create list of seasons
team_list   = dc['team'].unique()    ###Create list of teams
dm = pd.DataFrame()             ####Create blank data frame

for season in season_list:
    dt = dc[dc['Season'] == season]    ####Create data for one season
    sta_date = dt['Date'].min()
    end_date = dt['Date'].max()
    for team in team_list:
        du = pd.DataFrame(data=None, columns=['team'], index=pd.date_range(sta_date, end_date)).reset_index()
        du = du.rename(columns={'index': 'Date'})
        du['team'] = team
        dv = pd.DataFrame()
        dv = pd.merge(du, dt, left_on=['Date', 'team'], right_on=['Date', 'team'], how='left').set_index(['Season', 'team']) ###Data for one team for one season
        dm = pd.concat([dm, dv], axis=0)
print 'loop complete'

loop complete


# Insert team record for days with no games

In [233]:
dm = dm.reset_index(level=0)
dm['toffday'] = dm['tgameday'].apply(lambda x: 1 if x != 1 else 0)
dm = dm.fillna(method='bfill')

# Include game outcomes only from May 1 to the end of the season

In [234]:
dm['month'] = dm['Date'].apply(lambda x: x.month)
dm = dm[dm['month'] > 4]   ####Only leave May results to the end of the season results
dm = dm.reset_index()

# Create League Standings

In [263]:
dr = pd.DataFrame()
dr = dm[['Season', 'Date', 'League', 'Division', 'team', 'twin', 'tlos', 'gn', 'tws', 'tls', 'twper','Attendance','Streak1','PriorStreak']]
#dr = dr[dr['Season']==2014]
#dr = dr[(dr['League']=='AL') & (dr['Division']=='East')]
dr = dr.sort_values(by=['League', 'Division', 'Date', 'team', 'gn'], ascending=[1, 1, 1, 0, 1])
dr = dr.groupby(['League', 'Division', 'Date', 'team']).first().reset_index()

# Create League and Division Rank Variables

In [262]:
dr = dr.sort_values(by=['League', 'Division', 'Date', 'twper'], ascending=[1, 1, 1, 0])
dr['drank'] = dr.groupby(['League', 'Division', 'Date'])['twper'].rank(ascending = False)
dr = dr.sort_values(by=['League', 'Date', 'twper'], ascending=[1, 1, 0])
dr['lrank'] = dr.groupby(['League', 'Date'])['twper'].rank(ascending = False)

# Create data frame for Team and Opponent

In [237]:
dt = dr[['Season', 'Date', 'League', 'Division', 'team', 'drank', 'lrank']]
dt = dt.rename(columns={'drank': 'tdrank', 'lrank': 'tlrank'})
do = dr
do = do.rename(columns={'team': 'opp', 'tws': 'ows', 'tls': 'ols', 'twper': 'owper', 'drank': 'odrank', 'lrank': 'olrank', 'gn': 'ogn', 'Streak1': 'ostreak','PriorStreak':'opriorstreak'})


# Create data frame for Division and League Leader

In [238]:
dw = dr[dr['drank']==1][['Season', 'Date', 'League', 'Division', 'team', 'tws', 'tls', 'twper', 'twin', 'tlos']] ###division leader data
dw = dw.rename(columns={'team': 'dleader', 'tws': 'dlws', 'tls': 'dlls', 'twper': 'dlwper', 'twin': 'dltwin', 'tlos': 'dllos'})
dx = dr[dr['lrank']==1][['Season', 'Date', 'League', 'Division', 'team', 'tws', 'tls', 'twper', 'twin', 'tlos']] ####league leader data
dx = dx.rename(columns={'team': 'lleader', 'tws': 'llws', 'tls': 'llls', 'twper': 'llwper', 'twin': 'lltwin', 'tlos': 'lllos'})

# Create data set for analysis

In [239]:
da = pd.DataFrame()
da = dm[dm['tgameday']==1][['Season', 'Date', 'League', 'Division', 'team', 'opp', 'teamRuns', 'oppRuns', 'isthome', 'druns', 'twin', 'tws', 'tls', 'twper','DN','PriorStreak']]

# Merge Team, Opponent, Division leader, and League Leader data sets

In [240]:
da = pd.merge(da, dt, left_on=['Season', 'League', 'Division', 'Date', 'team'], right_on=['Season', 'League', 'Division', 'Date', 'team'])
da = pd.merge(da, do, left_on=['Season', 'Date', 'opp'] , right_on=['Season', 'Date', 'opp'])
da = pd.merge(da, dw, left_on=['Season', 'League_x', 'Division_x', 'Date'], right_on=['Season', 'League', 'Division', 'Date'])
da = pd.merge(da, dx, left_on=['Season', 'League_x', 'Date'], right_on=['Season', 'League', 'Date'])

# Create games back variables

In [241]:
'''playoff race indicative variables'''
da['gbll'] = (da['llws']-da['tws'] + da['tls']-da['llls'])/2
da['gbdl'] = (da['dlws']-da['tws'] + da['tls']-da['dlls'])/2

da = da.rename(columns={'League_x': 'tLeague', 'Division_x': 'tDivision', 'League_y': 'oLeague', 'Division_y': 'oDivision'})

# Attain previous day's game outcome for division leader

In [242]:
import datetime
from datetime import timedelta
dw['one_day'] = datetime.timedelta(days=1)
dw['nextday'] = dw['Date']+dw['one_day']
dw['Date'].dtypes

dtype('<M8[ns]')

# Merge previous division leader game outcome with team result

In [243]:
d1 = pd.merge(da, dw, left_on=['Season', 'Date', 'dleader'],right_on=['Season', 'nextday', 'dleader'])
d1.dtypes
d1.describe()
d1.groupby(level=0).first()
print 'merge complete'

merge complete


# Merge October results with September

In [244]:
d1['month'] = d1['Date_x'].apply(lambda x: x.month)
d1['month2'] = d1['month']
d1['month2'] = d1.apply(lambda x: 9 if (x['month2'] > 9) else x['month2'] , axis=1)

# Create variables representing games back by category

In [245]:
d1['gb'] = d1.apply(lambda x: 2 if (x['gbdl'] > 10) else x['gbdl'] , axis=1) #more than 10 games back
d1['gb'] = d1.apply(lambda x: 0 if (x['gbdl'] <= 5) else x['gb'] , axis=1) #5 to 10 games back
d1['gb'] = d1.apply(lambda x: 1 if (x['gbdl'] > 5) & (x['gbdl']<= 10) else x['gb'] , axis=1) #less than 5 games back

# Create data set that excludes division leader game outcomes

In [246]:
d1 = d1[d1['gbdl'] > 0]

# Convert variable types to floats

In [247]:
d1['Season']    = d1['Season'].astype(float)
d1['month']     = d1['month'].astype(float)
d1['gb']        = d1['gb'].astype(float)
d1['dltwin_x']  = d1['dltwin_x'].astype(float)
d1['twin_x']    = d1['twin_x'].astype(float)
d1['month2']    = d1['month2'].astype(float)
d1['PriorStreak']    = d1['PriorStreak'].astype(float)
d1['opriorstreak']    = d1['opriorstreak'].astype(float)

# Table 1: Summary Statistics of Scoreboard Watching

In [248]:
d3 = d1.groupby(['month2', 'gb', 'dltwin_x'])['twin_x'].mean()
print d3
d4 = d1.groupby(['month2', 'gb', 'dltwin_x'])['twin_x'].std()
print d4
print 'gb = 0: less than 5 games back'
print 'gb = 1: 5 to 10 games back'
print 'gb = 2: greater than 10 games back'

month2  gb   dltwin_x
5.0     0.0  0.0         0.566518
             1.0         0.473811
        1.0  0.0         0.494764
             1.0         0.451777
        2.0  0.0         0.533724
                           ...   
9.0     0.0  1.0         0.506045
        1.0  0.0         0.625187
             1.0         0.454128
        2.0  0.0         0.542974
             1.0         0.380597
Name: twin_x, dtype: float64
month2  gb   dltwin_x
5.0     0.0  0.0         0.495713
             1.0         0.499464
        1.0  0.0         0.500191
             1.0         0.497880
        2.0  0.0         0.499594
                           ...   
9.0     0.0  1.0         0.500396
        1.0  0.0         0.484438
             1.0         0.498177
        2.0  0.0         0.498263
             1.0         0.485624
Name: twin_x, dtype: float64
gb = 0: less than 5 games back
gb = 1: 5 to 10 games back
gb = 2: greater than 10 games back


# Create additional variables for analysis

In [249]:
d1['lngbdl']    = np.log(d1['gbdl']) #log of games back
d1['dwper']     = d1['twper'] - d1['owper'] #difference in winning percentage with opponent
d1['dlw_gbdl']  = d1['dltwin_x'] * d1['gbdl'] #games back of division leader if division leader won
d1['dll_gbdl']  = (1 - d1['dltwin_x']) * d1['gbdl'] #games back of division leader if division leader lost

d1['twperhalf'] = d1['twper']-0.5 #team winning percentage minus 50%

Y = d1['twin_x'] #Create dependent variable

# Table 2: Ordinary Least Squares in relation to Scoreboard Watching and Win Probability

In [250]:
print "all seasons"
print 'all months'
X = sm.add_constant(d1[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'May to August'
dt = d1[d1['month2']<9]
Y = dt['twin_x']
X = sm.add_constant(dt[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'September to October'
ds = d1[d1['month2']>=9]
Y = ds['twin_x']
X = sm.add_constant(ds[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Teams still in race during September/October'
ds['gr'] = 162-ds['tws']-ds['tls']
dsi = ds[ds['gr']>=ds['gbdl']]

Y = dsi['twin_x']
X = sm.add_constant(dsi[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Teams still in race during September/October based on games back'
Y = dsi['twin_x']
X = sm.add_constant(dsi[['dltwin_x', 'dwper', 'dlw_gbdl', 'dll_gbdl']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()



all seasons
all months
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     368.8
Date:                Thu, 08 Dec 2016   Prob (F-statistic):          3.07e-159
Time:                        18:21:19   Log-Likelihood:                -25309.
No. Observations:               35386   AIC:                         5.062e+04
Df Residuals:                   35383   BIC:                         5.065e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.5547      0.

/home/vmuser/local/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Table 3: Scoreboard Watching Regression Based on Playoff Races

In [251]:
print 'August in race'
da = d1[d1['month2']==8]
da = da[da['gbdl']<=5]
Y = da['twin_x']
X = sm.add_constant(da[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'September in race'
dsi5 = dsi[dsi['gbdl']<=5]
Y = dsi5['twin_x']
X = sm.add_constant(dsi5[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

August in race
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     12.18
Date:                Thu, 08 Dec 2016   Prob (F-statistic):           5.65e-06
Time:                        18:21:20   Log-Likelihood:                -1141.5
No. Observations:                1590   AIC:                             2289.
Df Residuals:                    1587   BIC:                             2305.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.5473      0.021     

# Table 4: Scoreboard Watching Regression with Team Fixed Effects

In [252]:
print 'September in race with team fixed effects'
namesList = pd.get_dummies(dsi5['team'], prefix='TX') #indicator variables
dsi5 = dsi5.join(namesList) #merge with dsi5

Y = dsi5['twin_x']
X = sm.add_constant(dsi5[['dltwin_x', 'dwper', 'TX_ATL','TX_BAL', 'TX_BOS','TX_CIN','TX_CLE','TX_COL','TX_CHW','TX_DET','TX_HOU','TX_KCR','TX_LAA','TX_LAD','TX_MIA','TX_MIL','TX_MIN','TX_NYM','TX_NYY','TX_OAK','TX_PHI','TX_PIT','TX_SDP','TX_SFG','TX_STL','TX_TBR','TX_TEX','TX_WSN']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()


September in race with team fixed effects
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.015
Method:                 Least Squares   F-statistic:                     1.539
Date:                Thu, 08 Dec 2016   Prob (F-statistic):             0.0371
Time:                        18:21:20   Log-Likelihood:                -692.54
No. Observations:                 995   AIC:                             1443.
Df Residuals:                     966   BIC:                             1585.
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const     

# Table 5: Scoreboard Watching Regression Based on Home/Away

In [253]:
print 'September in race before 2012'
db2012 = dsi5[dsi5['Season']<2012]
Y = db2012['twin_x']
X = sm.add_constant(db2012[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'September in race 2012 to 2014'
da2012 = dsi5[dsi5['Season']>=2012]
Y = da2012['twin_x']
X = sm.add_constant(da2012[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

September in race before 2012
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     5.407
Date:                Thu, 08 Dec 2016   Prob (F-statistic):            0.00466
Time:                        18:21:20   Log-Likelihood:                -527.04
No. Observations:                 741   AIC:                             1060.
Df Residuals:                     738   BIC:                             1074.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.6035 

# Table 6: Scoreboard Watching Regression Based on Playoff Structure

In [254]:
print 'September in race: home team'
dhome = dsi5[dsi5['isthome']==1]
Y = dhome['twin_x']
X = sm.add_constant(dhome[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'September in race: away team'
daway = dsi5[dsi5['isthome']==0]
Y = daway['twin_x']
X = sm.add_constant(daway[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

September in race: home team
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     15.07
Date:                Thu, 08 Dec 2016   Prob (F-statistic):           4.49e-07
Time:                        18:21:20   Log-Likelihood:                -333.52
No. Observations:                 487   AIC:                             673.0
Df Residuals:                     484   BIC:                             685.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.6670  

# Table 7: Scoreboard Watching Regression Based on Games Remaining

In [255]:
print 'September in race with more than 20 games remaining'
d20 = dsi5[dsi5['gr']>20]
Y = d20['twin_x']
X = sm.add_constant(d20[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'September in race with more than 10 and less than 20 games remaining'
d10 = dsi5[dsi5['gr']>10]
d10 = d10[d10['gr']<=20]
Y = d10['twin_x']
X = sm.add_constant(d10[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'September in race with 10 or fewer games remaining'
dend = dsi5[dsi5['gr']<=10]
Y = dend['twin_x']
X = sm.add_constant(dend[['dltwin_x', 'dwper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

September in race with more than 20 games remaining
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.128
Date:                Thu, 08 Dec 2016   Prob (F-statistic):              0.325
Time:                        18:21:20   Log-Likelihood:                -215.40
No. Observations:                 303   AIC:                             436.8
Df Residuals:                     300   BIC:                             447.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------


# Table 8: Impact of Roster’s Average Age on Scoreboard Watching

In [256]:
print 'Regression based on Average Age'
dsage = pd.merge(dsi5,dage, left_on=['team', 'Season'], right_on=['Tm', 'Season'])
dsavg = pd.merge(dsage,dage, left_on=['opp', 'Season'], right_on=['Tm', 'Season'])
dsavg['diffage'] = dsavg['AvgAge_x'] - dsavg['AvgAge_y']
dsavg['intdiffage'] = dsavg['diffage']* dsavg['dltwin_x']
dsavg['diffagebat'] = dsavg['BatAge_x'] - dsavg['BatAge_y']
dsavg['intdiffagebat'] = dsavg['diffagebat']* dsavg['dltwin_x']
dsavg['diffagepit'] = dsavg['PAge_x'] - dsavg['PAge_y']
dsavg['intdiffagepit'] = dsavg['diffagepit']* dsavg['dltwin_x']

Y = dsavg['twin_x']
X = sm.add_constant(dsavg[['dltwin_x', 'dwper', 'intdiffage']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Based on Position Age'
Y = dsavg['twin_x']
X = sm.add_constant(dsavg[['dltwin_x', 'dwper', 'intdiffagebat', 'intdiffagepit']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Logs of difference in age'
dsavg['lnintdiffage']= np.log(dsavg['AvgAge_x']/dsavg['AvgAge_y'])* dsavg['dltwin_x']
Y = dsavg['twin_x']
X = sm.add_constant(dsavg[['dltwin_x', 'dwper', 'lnintdiffage']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Logs of difference in age by position'
dsavg['lnintdiffagebat']= np.log(dsavg['BatAge_x']/dsavg['BatAge_y'])* dsavg['dltwin_x']
dsavg['lnintdiffagepit']= np.log(dsavg['PAge_x']/dsavg['PAge_y'])* dsavg['dltwin_x']
Y = dsavg['twin_x']
X = sm.add_constant(dsavg[['dltwin_x', 'dwper', 'lnintdiffagebat', 'lnintdiffagepit']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

Regression based on Average Age
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     6.522
Date:                Thu, 08 Dec 2016   Prob (F-statistic):           0.000228
Time:                        18:21:20   Log-Likelihood:                -704.52
No. Observations:                 995   AIC:                             1417.
Df Residuals:                     991   BIC:                             1437.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.601

# Table 9: Impact of Team Payroll on Scoreboard Watching

In [257]:
dspay = pd.merge(dsi5,dpay, left_on=['team', 'Season'], right_on=['Team', 'Year'])
dsal = pd.merge(dspay,dpay, left_on=['opp', 'Season'], right_on=['Team', 'Year'])
dsal['diffsal'] = dsal['End Pay_x'] - dsal['End Pay_y']
dsal['intdiffsal'] = dsal['diffsal']* dsal['dltwin_x']
dsal['diffrank'] = dsal['End Rank_x'] - dsal['End Rank_y']
dsal['intdiffrank'] = dsal['diffrank']* dsal['dltwin_x']
dsal['lnintdiffsal']= np.log(dsal['End Pay_x']/dsal['End Pay_y'])* dsal['dltwin_x']
dsal['lnintdiffrank']= np.log(dsal['End Rank_x']/dsal['End Rank_y'])* dsal['dltwin_x']

print 'Interaction of scoreboard watching with team payroll'
Y = dsal['twin_x']
X = sm.add_constant(dsal[['dltwin_x', 'dwper', 'lnintdiffsal']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Interaction of scoreboard watching with team payroll rank'
Y = dsal['twin_x']
X = sm.add_constant(dsal[['dltwin_x', 'dwper', 'lnintdiffrank']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

Interaction of scoreboard watching with team payroll
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     6.532
Date:                Thu, 08 Dec 2016   Prob (F-statistic):           0.000224
Time:                        18:21:20   Log-Likelihood:                -704.51
No. Observations:                 995   AIC:                             1417.
Df Residuals:                     991   BIC:                             1437.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------

# Table 10: Impact of Managerial Experience on Scoreboard Watching

In [258]:
dman = pd.merge(dsi5,dmgr, left_on=['team', 'Season'], right_on=['Team', 'Season'])
dmanager = pd.merge(dman,dmgr, left_on=['opp', 'Season'], right_on=['Team', 'Season'])
dmanager['diffplayoff'] = dmanager['Prior Playoff_x'] - dmanager['Prior Playoff_y']
dmanager['intdiffplayoff'] = dmanager['diffplayoff']* dmanager['dltwin_x']
dmanager['diffseasons'] = dmanager['Prior Seasons Managed_x'] - dmanager['Prior Seasons Managed_y']
dmanager['intdiffseasons'] = dmanager['diffseasons']* dmanager['dltwin_x']
dmanager['diffplayper'] = dmanager['playoff per season_x'] - dmanager['playoff per season_y']
dmanager['intdiffplayper'] = dmanager['diffplayper']* dmanager['dltwin_x']

print 'Interaction of scoreboard watching with playoff seasons'
Y = dmanager['twin_x']
X = sm.add_constant(dmanager[['dltwin_x', 'dwper', 'intdiffplayoff']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Interaction of scoreboard watching with seasons managed'
Y = dmanager['twin_x']
X = sm.add_constant(dmanager[['dltwin_x', 'dwper', 'intdiffseasons']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Interaction of scoreboard watching with percent of seasons in playoffs'
Y = dmanager['twin_x']
X = sm.add_constant(dmanager[['dltwin_x', 'dwper', 'intdiffplayper']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

Interaction of scoreboard watching with playoff seasons
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     6.571
Date:                Thu, 08 Dec 2016   Prob (F-statistic):           0.000212
Time:                        18:21:20   Log-Likelihood:                -706.42
No. Observations:                 997   AIC:                             1421.
Df Residuals:                     993   BIC:                             1440.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------

# Table 11: Impact of Momentum on Scoreboard Watching

In [259]:
dsi5['diffstreak'] = dsi5['PriorStreak'] - dsi5['opriorstreak']
dsi5 ['intdiffstreak'] = dsi5['diffstreak']* dsi5['dltwin_x']
dwstreak = dsi5[dsi5['PriorStreak'] > 0]
dlstreak = dsi5[dsi5['PriorStreak'] < 0]

print 'Interaction of scoreboard watching with Streaks'
Y = dsi5['twin_x']
X = sm.add_constant(dsi5[['dltwin_x', 'dwper', 'intdiffstreak']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Interaction of scoreboard watching with Winning Streaks'
Y = dwstreak['twin_x']
X = sm.add_constant(dwstreak[['dltwin_x', 'dwper', 'intdiffstreak']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Interaction of scoreboard watching with Losing Streaks'
Y = dlstreak['twin_x']
X = sm.add_constant(dlstreak[['dltwin_x', 'dwper', 'intdiffstreak']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

Interaction of scoreboard watching with Streaks
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     6.713
Date:                Thu, 08 Dec 2016   Prob (F-statistic):           0.000174
Time:                        18:21:20   Log-Likelihood:                -704.24
No. Observations:                 995   AIC:                             1416.
Df Residuals:                     991   BIC:                             1436.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------

# Table 12: Impact of Attendance on Scoreboard Watching

In [261]:
dsi5 = dsi5[dsi5['Attendance'] > 1] #include attennded games only (First game of double header has no recorded attendance)

dsi5['Attendance']    = dsi5['Attendance'].astype(float)
dsi5['IntAttendance'] = ((dsi5['Attendance'])/1000)
dsi5['lnAttendance']= (np.log(dsi5['Attendance']))

print 'Interaction of scoreboard watching with Attendance'
Y = dsi5['twin_x']
X = sm.add_constant(dsi5[['dltwin_x', 'dwper', 'IntAttendance']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

print 'Interaction of scoreboard watching with Log of Attendance'
Y = dsi5['twin_x']
X = sm.add_constant(dsi5[['dltwin_x', 'dwper', 'lnAttendance']])
tempOut = sm.OLS(Y, X).fit()
print tempOut.summary()

Interaction of scoreboard watching with Attendance
                            OLS Regression Results                            
Dep. Variable:                 twin_x   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     6.388
Date:                Thu, 08 Dec 2016   Prob (F-statistic):           0.000275
Time:                        19:44:07   Log-Likelihood:                -695.81
No. Observations:                 982   AIC:                             1400.
Df Residuals:                     978   BIC:                             1419.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------